# Projet : Mise en œuvre d'un modèle de Machine Learning avec TensorFlow

Ce notebook couvre toutes les étapes nécessaires pour développer un modèle de détection de fraude par carte bancaire, depuis le traitement des données jusqu'à l'évaluation du modèle, conformément aux exigences du projet.

## Chargement des données

Nous utilisons le dataset `Credit Card Fraud Detection` disponible sur Kaggle.

In [ ]:

import pandas as pd

# Chargement des données
data = pd.read_csv('creditcard.csv')
print(data.head())
print(data.info())


## Préparation des données

### Nettoyage des données et exploration initiale

In [ ]:

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Normalisation des données numériques
scaler = StandardScaler()
data['scaled_amount'] = scaler.fit_transform(data[['Amount']])
data['scaled_time'] = scaler.fit_transform(data[['Time']])
data = data.drop(['Time', 'Amount'], axis=1)

# Exploration rapide des classes
fraud = data[data['Class'] == 1]
non_fraud = data[data['Class'] == 0]

print("Nombre de transactions frauduleuses :", len(fraud))
print("Nombre de transactions non frauduleuses :", len(non_fraud))

# Visualisation
plt.figure(figsize=(8, 5))
data['Class'].value_counts().plot(kind='bar', title='Distribution des classes (fraude vs non-fraude)')
plt.show()


### Gestion du déséquilibre des classes avec SMOTE

In [ ]:

from imblearn.over_sampling import SMOTE

# Séparation des caractéristiques et étiquettes
X = data.drop('Class', axis=1)
y = data['Class']

# Application de SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Distribution après SMOTE :")
print(y_resampled.value_counts())


## Mise en place de l'architecture du modèle

Créons un réseau de neurones avec TensorFlow.

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Création du modèle
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_resampled.shape[1],)),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


## Entraînement du modèle

Nous entraînons le modèle sur l'ensemble d'entraînement et utilisons la validation croisée.

In [ ]:

from sklearn.model_selection import train_test_split

# Séparation des ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Entraînement
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)


## Évaluation du modèle

Calcul des métriques de performance et visualisation des résultats.

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Prédictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Rapport de classification
print(classification_report(y_test, y_pred))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Matrice de Confusion")
plt.show()
